### Importing Required Libraries

In [33]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

### Reading Data from Excel

In [34]:
data = pd.read_csv('CarPrice_Assignment.csv')
data = data.set_index('car_ID')

In [107]:
encoding = LabelEncoder()

data["fueltype"] = encoding.fit_transform(data["fueltype"])
data["aspiration"] = encoding.fit_transform(data["aspiration"])
data["doornumber"] = encoding.fit_transform(data["doornumber"])
data["carbody"] = encoding.fit_transform(data["carbody"])
data["drivewheel"] = encoding.fit_transform(data["drivewheel"])
data["enginetype"] = encoding.fit_transform(data["enginetype"])
data["cylindernumber"] = encoding.fit_transform(data["cylindernumber"])
data["enginelocation"] = encoding.fit_transform(data["enginelocation"])
data["fuelsystem"] = encoding.fit_transform(data["fuelsystem"])

### Preprocessing Data

In [143]:
# Converting to numpy
X = data.iloc[:,2:24].to_numpy(copy=True)
Y = data['price'].to_numpy(copy=True)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

# Normalizing
sc = StandardScaler(); minmax = MinMaxScaler()
X_train = sc.fit_transform(X_train); X_test = sc.transform(X_test)
y_train = minmax.fit_transform(y_train.reshape(-1,1)); y_test = minmax.transform(y_test.reshape(-1,1))
X_train = X_train.astype('float32'); X_test = X_test.astype('float32')
y_train = y_train.astype('float32'); y_test = y_test.astype('float32')

In [144]:
batch_size = 8
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = train_data.shuffle(buffer_size=len(y_train)).batch(batch_size)

### Creating Model

In [131]:
ep = 100; # epoch number
e_1 = 0.2; e_2 = 0.1
l_1 = 0.7; l_2 = 0.9; l_3 = 1 

In [132]:
model = keras.models.Sequential([
    keras.layers.Dense(50, activation = "relu"),
    keras.layers.Dense(50, activation = "relu"),
    keras.layers.Dense(50, activation = "relu"),
    keras.layers.Dense(1,  activation = "tanh")
])

class My_loss(keras.losses.Loss):
     
    def __init__(self, e1, e2, l1, l2, l3, **kwargs):
        self.e1 = e1; self.e2 = e2
        self.l1 = l1; self.l2 = l2; self.l3 = l3
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = tf.abs(y_true - y_pred)
        return tf.experimental.numpy.select([error <= self.e2,
                                             error < self.e1,
                                             error >= self.e1], 
                                            [self.l3*error, self.l2*error, self.l1*error])
    def get_config(self):
        parent_config = super().get_config()
        return {**parent_config, "e1":self.e1, "e2":self.e2, "l1":self.l1, "l2":self.l2, "l3":self.l3}

In [133]:
e_1 = 0.2; e_2 = 0.1; l_1 = 0.7; l_2 = 0.9; l_3 = 1 
Loss_fn = My_loss(e1=e_1, e2=e_2, l1=l_1, l2=l_2, l3=l_3)

In [138]:
Optimizer = tf.keras.optimizers.Adam()
Metric = tf.keras.metrics.MeanAbsoluteError()
Metric_test = tf.keras.metrics.MeanAbsoluteError()

In [142]:
epochs = 50

for epoch in range(epochs):
    
    loss_train_epoch_sum = tf.constant(0,dtype='float32');
    
    for batch_i, (x_train, y_train) in enumerate(train_data):
        
        with tf.GradientTape() as g:
            model_out = model(x_train, training=True)  
            loss_train_batch = Loss_fn(y_train, model_out)
        
        loss_train_epoch_sum += loss_train_batch

        grads = g.gradient(loss_train_batch, model.trainable_weights)

        Optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        
    model_out_validation = model(X_test, training=False)
    loss_test_epoch = Loss_fn(y_test, model_out_validation)
    Metric_test.update_state(y_test, model_out_validation)
    
    loss_train_epoch = loss_train_epoch_sum/len(train_data)
    Metric.update_state(y_train, model_out)

    if epoch % 10 == 0:
        print("Training Loss:", float(loss_train_epoch))
        print("Validation Loss:", float(loss_test_epoch))
        
        Metric_train = Metric.result()
        print("Train_Metric_Value:", float(Metric_train))
        
        Metric_validation = Metric_test.result()
        print("Validation_Metric_Value:", float(Metric_validation),'\n')

    Metric.reset_states()
    Metric_test.reset_states()

Training Loss: 0.014580439776182175
Validation Loss: 0.043052151799201965
Train_Metric_Value: 0.028096787631511688
Validation_Metric_Value: 0.045986611396074295 

Training Loss: 0.013614709489047527
Validation Loss: 0.03989577665925026
Train_Metric_Value: 0.013899044133722782
Validation_Metric_Value: 0.04284137487411499 

Training Loss: 0.016608314588665962
Validation Loss: 0.04150553420186043
Train_Metric_Value: 0.033851925283670425
Validation_Metric_Value: 0.04453856870532036 

Training Loss: 0.014383713714778423
Validation Loss: 0.041212499141693115
Train_Metric_Value: 0.009930362924933434
Validation_Metric_Value: 0.04460093006491661 

Training Loss: 0.0165505800396204
Validation Loss: 0.04056519269943237
Train_Metric_Value: 0.022154681384563446
Validation_Metric_Value: 0.04378392919898033 

